In [34]:
from calculating_risk_for_single_community import calculating_risk_for_single_community as single_calculate
import pandas as pd
import numpy as np
from ast import literal_eval

population = pd.read_csv('Enriched_LA_CSA_0.csv')
infection = pd.read_csv('county_storage.csv')

In [35]:
infection['location'] = infection.apply(lambda x: x['location'].replace('UNINCORPORATED - ','').lower().replace('city of ',''), axis = 1)
infection

,Date,case_rate,cases,death_rate,deaths,location
0,04/15/2020,100.56,21,NaN,NaN,agoura hills
1,04/15/2020,42.66,37,NaN,NaN,alhambra
2,04/15/2020,43.29,25,NaN,NaN,arcadia
3,04/15/2020,29.77,5,NaN,NaN,artesia
4,04/15/2020,0,0,NaN,NaN,avalon
...,...,...,...,...,...,...
25531,07/06/2020,597,22,0,0,white fence farms
25532,07/06/2020,766,29,0,0,whittier
25533,07/06/2020,0,0,0,0,whittier narrows
25534,07/06/2020,1876,655,29,10,willowbrook


In [36]:
population2 = pd.read_csv('processed_population.csv')
population2

,Region,Latitude,Longitude,Population
0,Northridge,34.234561,-118.536932,67755.0
1,Exposition Park,34.013654,-118.287211,45190.0
2,Norwalk,33.909280,-118.084917,104868.0
3,Hawthorne,33.918859,-118.348326,114032.0
4,El Segundo,33.917028,-118.415634,26864.0
...,...,...,...,...
163,Claremont,34.096676,-117.719779,36850.0
164,Tarzana,34.171444,-118.542979,30227.0
165,South Park,34.040407,-118.266538,39093.0
166,Eagle Rock,34.138897,-118.213962,39175.0


In [58]:
count = 0
for index, row in population2.iterrows():
    popl = row['Population']
    loc = row['Region']
    data = infection[infection['location'] == loc.lower()]
    if len(data)>0:
        count += 1
        #if count<=78:
        #    continue
        vec = list(data['cases'])
        vec = np.array(list(filter(lambda a: a != '--', vec)),dtype = 'float')
        risk_score = single_calculate(vec, popl, 1/7.5, 1)
        model_results = pd.DataFrame([{'location':loc,'population':popl,'cases':list(vec),'risk_score':list(risk_score),'last_date':max(data['Date'])}])
        model_results.to_csv('retrain_model_results_USC.csv', mode = 'a', index = False, header = False)
        print (loc+'      '+str(count))


San Gabriel      79
Lancaster      80
Pico Rivera      81
Gardena      82
Claremont      83


In [86]:
def cal_error(n_ahead, case, risk, popl):
    n = len(case)
    new_infe = []
    risk_infe = []
    for i in range(n-n_ahead):
        new_infe.append(case[i+n_ahead]-case[i])
        risk_infe.append(risk[i])
    new_infe = np.array(new_infe) / np.linalg.norm(new_infe)
    risk_infe = np.array(risk_infe) / np.linalg.norm(risk_infe)
    return np.linalg.norm(new_infe-risk_infe)

In [89]:
from ast import literal_eval

retrain = pd.read_csv('retrain_model_results_USC.csv')
retrain[['cases', 'risk_score']] = retrain[['cases', 'risk_score']].applymap(literal_eval)
retrain['error_3'] = 0
retrain['error_7'] = 0
for index, row in retrain.iterrows():
    pop = row['locatpopulationion']
    case = np.array(row['cases'], dtype = 'float')
    risk = np.array(row['risk_score'], dtype = 'float')
    if sum(risk)==0:
        continue
    retrain.iloc[index, retrain.columns.get_loc('error_3')] = cal_error(3, case, risk, pop)
    retrain.iloc[index, retrain.columns.get_loc('error_7')] = cal_error(7, case, risk, pop)


In [90]:
retrain

,location,locatpopulationion,cases,risk_score,last_date,error_3,error_7
0,Norwalk,104868.0,"[88.0, 90.0, 93.0, 100.0, 105.0, 124.0, 135.0,...","[9.81418798861426, 10.702125158985583, 13.7077...",07/06/2020,0.436984,0.315303
1,Hawthorne,114032.0,"[87.0, 90.0, 89.0, 92.0, 94.0, 134.0, 140.0, 1...","[9.576742147519994, 7.240525881588877, 9.75317...",07/06/2020,0.788814,0.777411
2,El Segundo,26864.0,"[12.0, 12.0, 12.0, 14.0, 14.0, 21.0, 24.0, 25....","[4.46894086101594, 4.46894086101594, 9.6359674...",07/06/2020,0.802639,0.687968
3,Glendale,201605.0,"[278.0, 288.0, 310.0, 326.0, 332.0, 369.0, 389...","[17.468658563349123, 22.205062227782047, 21.21...",07/06/2020,0.546895,0.494056
4,Huntington Park,59508.0,"[44.0, 46.0, 52.0, 56.0, 58.0, 87.0, 95.0, 106...","[9.866484092458158, 14.850184205535394, 13.607...",07/06/2020,0.442825,0.324408
...,...,...,...,...,...,...,...
78,San Gabriel,40311.0,"[14.0, 15.0, 15.0, 16.0, 16.0, 23.0, 25.0, 26....","[5.2719835416759695, 3.7224538415723645, 5.524...",07/06/2020,0.622193,0.532413
79,Lancaster,169552.0,"[91.0, 96.0, 102.0, 114.0, 120.0, 184.0, 197.0...","[7.524320747303188, 8.2412056681608, 11.041255...",07/06/2020,0.500103,0.407954
80,Pico Rivera,64052.0,"[67.0, 73.0, 77.0, 79.0, 81.0, 107.0, 113.0, 1...","[17.207613759414222, 15.975568742439735, 14.35...",07/06/2020,0.490163,0.395153
81,Gardena,60679.0,"[82.0, 86.0, 93.0, 96.0, 102.0, 121.0, 126.0, ...","[18.366401729066066, 22.524125526409467, 19.00...",07/06/2020,0.456325,0.325679


In [91]:
retrain['error_3'].mean()

0.5784364124018813

In [92]:
retrain['error_7'].mean()

0.48176764175962555